In [ ]:
%pip install datasets
%pip install Pillow
%pip install numpy
%pip install matplotlib
%pip install import_ipynb
%pip install tensorflow
%pip install tensorflow.keras
%pip install easyocr

# library from etl
%pip install datasets
%pip install numpy
%pip install Pillow
%pip install opencv-python
%pip install json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [ ]:
import easyocr
import matplotlib.pyplot as plt
from datasets import load_dataset
import numpy as np
from PIL import Image
import cv2
import tensorflow as tf
# from tensorflow.keras import layers
import json

In [ ]:
from google.colab import userdata

# Retrieve the GitHub token from userdata
myGithub = userdata.get('githubKey')

!git clone https://{myGithub}@github.com/fitristachan/invoice-image-processing.git /content/invoice_image_processing
!git -C invoice_image_processing pull



In [ ]:
# import sys
# import os

# sys.path.append('/content/invoice-image-processing')
# print(os.listdir('/content/invoice-image-processing'))

# !jupyter nbconvert --to script '/content/invoice-image-processing/data_etl_invoice.ipynb' --output data_etl

Image Classification

In [ ]:
from invoice_image_processing.data_etl import DatasetReceipt

In [ ]:
# Inisialisasi dataset
dataset_local = DatasetReceipt(dataset_name="naver-clova-ix/cord-v2", split="train")
dataset_inter = DatasetReceipt(dataset_name="katanaml-org/invoices-donut-data-v1", split="train")

In [ ]:
from keras.applications import ResNet50V2

# keras.applications.ResNet50V2(
#     include_top=True,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
#     classifier_activation="softmax",
#     name="resnet50v2",
# )

pre_trained_model = ResNet50V2(input_shape = (600, 600, 3),
                                include_top = False,
                                pooling='max'
                                )

pre_trained_model.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load Dataset (Gunakan DatasetReceipt class yang sudah dibuat sebelumnya)
train_dataset = DatasetReceipt(dataset_name="naver-clova-ix/cord-v2", split="train")
# val_dataset = DatasetReceipt(dataset_name="katanaml-org/invoices-donut-data-v1", split="train")
val_dataset = DatasetReceipt(dataset_name="naver-clova-ix/cord-v2", split="validation")

# Function to generate batches
def dataset_generator(dataset, batch_size=16, max_items=10):
    while True:
        batch_images = []
        batch_bboxes = []
        batch_classes = []
        batch_prices = []
        batch_totals = []

        for _ in range(batch_size):
            sample = dataset[np.random.randint(len(dataset))]

            # 1. Image (remove batch dimension if present)
            img = sample["image"]
            if img.ndim == 4 and img.shape[0] == 1:
                img = img[0]
            batch_images.append(img)

            # 2. Bounding boxes (mock data - replace with real coordinates)
            batch_bboxes.append([0.1, 0.1, 0.5, 0.5])  # [x1,y1,x2,y2]

            # 3. Classes (one-hot encoded mock data)
            batch_classes.append(np.eye(5)[0])  # 5 classes

            # 4. Prices (handle missing/negative values)
            prices = sample["menu_df"]["price"].values
            prices = np.clip(prices, 0, None)  # Replace negatives with 0
            padded_prices = np.zeros(max_items)
            padded_prices[:min(len(prices), max_items)] = prices[:max_items]
            batch_prices.append(padded_prices)

            # 5. Total (ensure positive)
            total = max(0, sample["total_price"])
            batch_totals.append([total])

        yield (
            np.array(batch_images),
            {
                "bounding_box": np.array(batch_bboxes),
                "class_label": np.array(batch_classes),
                "price_regression": np.array(batch_prices),
                "total_regression": np.array(batch_totals)
            }
        )

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.applications import ResNet50V2

# 1. Define Input Layer
inputs = Input(shape=(600, 600, 3))  # Match your image size

# 2. Load Pretrained ResNet50V2 (without top)
base_model = ResNet50V2(
    input_shape=(600, 600, 3),
    include_top=False,
    pooling='max'  # GlobalMaxPooling2D
)
base_model.trainable = False  # Freeze weights

# 3. Connect Base Model to Inputs
x = base_model(inputs)

# 4. Add Custom Layers
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

# 5. Define Output Layers
bbox_output = Dense(4, name="bounding_box")(x)  # 4 coordinates
class_output = Dense(5, activation='softmax', name="class_label")(x)
price_output = Dense(10, name="price_regression")(x)  # Predict 10 prices
total_output = Dense(1, name="total_regression")(x)  # Predict total

# 6. Create Model
model = Model(
    inputs=inputs,
    outputs=[bbox_output, class_output, price_output, total_output]
)

# 7. Compile
model.compile(
    optimizer='adam',
    loss={
        "bounding_box": "mse",
        "class_label": "categorical_crossentropy",
        "price_regression": "mse",
        "total_regression": "mse"
    },
    metrics={"total_regression": "mae"}
)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

checkpoint_path = "/content/drive/MyDrive/dataset/model/model_checkpoint_resnet50v2.h5"
checkpoint_callback = ModelCheckpoint(checkpoint_path, save_best_only=False)

In [ ]:
class Callbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy'] > 0.95 and logs['val_accuracy'] > 95:
            print("\nAccuracy reached 95%! Stopping training.")
            self.model.stop_training = True

stop_callback = Callbacks()

In [ ]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('training_log.csv', separator=',', append=True)

In [ ]:
# Data Augmentation
train_generator = dataset_generator(train_dataset, batch_size=16)
val_generator = dataset_generator(val_dataset, batch_size=16)

In [ ]:
# # Train Model
# model.fit(train_generator,
#           validation_data=val_generator,
#           epochs=25,
#           steps_per_epoch=1000,
#           validation_steps=200,
#           callbacks=[checkpoint_callback, csv_logger])

model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,
    steps_per_epoch=100,
    validation_steps=50
)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load training log
log_data = pd.read_csv("training_log.csv")

# Buat figure untuk dua grafik
plt.figure(figsize=(16, 6))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.title("Model Accuracy")
plt.plot(log_data["epoch"], log_data["bounding_box_mae"], label="Training MAE")
plt.plot(log_data["epoch"], log_data["val_bounding_box_mae"], label="Validation MAE")
plt.plot(log_data["epoch"], log_data["class_label_accuracy"], label="Training Accuracy")
plt.plot(log_data["epoch"], log_data["val_class_label_accuracy"], label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy / MAE")
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.title("Model Loss")
plt.plot(log_data["epoch"], log_data["loss"], label="Training Loss")
plt.plot(log_data["epoch"], log_data["val_loss"], label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.show()

In [ ]:
# # Save Model
# model.save("faster_rcnn_resnet50v2.h5")
# print("Model saved successfully!")

**Text Extraction**

> using easyocr

> support bahasa indonesia and english





In [ ]:
reader = easyocr.Reader(['en', 'id'])

def is_receipt(text):
    text_lower = text.lower()

    if 'total' or 'amount' or 'jumlah' and 'price' or 'harga' in text_lower:
        print("This is a valid receipt.")
    else:
        print("This is not a valid receipt.")


def process_image(image):
    # Convert PIL Image to numpy array
    image_np = np.array(image)
    if image_np.dtype != np.uint8:
        image_np = (image_np * 255).astype(np.uint8)  # Normalisasi jika float64

    # Extract text using EasyOCR
    results = reader.readtext(image_np)

    # Combine all detected text into a single string
    extracted_text = ' '.join([result[1] for result in results])

    # Check if the extracted text is a receipt
    is_receipt(extracted_text)

    print("Extracted Text:")
    print(extracted_text)

Fine Tuning

In [ ]:
# data_tuning_local = load_dataset("naver-clova-ix/cord-v2", split="train")
# data_tuning_inter = load_dataset("katanaml-org/invoices-donut-data-v1", split="train")
# data_tuning_inter[5]


In [ ]:
# import os
# import json

# def convert_dataset_local(dataset, output_file):
#     annotations = []

#     # Pastikan folder untuk menyimpan gambar ada
#     os.makedirs("dataset_images", exist_ok=True)

#     for i, data in enumerate(dataset):
#         image = data["image"]  # Objek gambar dari PIL
#         ground_truth = json.loads(data["ground_truth"])  # Parsing JSON string

#         # Simpan gambar ke folder agar bisa digunakan oleh EasyOCR
#         image_path = f"dataset_images/image_{i}.jpg"
#         image.save(image_path)

#         # Ambil teks dan bounding box dari ground truth
#         text = ground_truth["gt_parse"]  # Semua teks dan informasi struk
#         words = []
#         for line in ground_truth["valid_line"]:  # Ambil setiap baris
#             for word in line["words"]:  # Ambil setiap kata dalam baris
#                 words.append({
#                     "text": word["text"],
#                     "quad": word["quad"],
#                     "row_id": word["row_id"],
#                     "category": line["category"],
#                     "group_id": line["group_id"]
#                 })

#         # Simpan dalam format yang sesuai untuk EasyOCR
#         annotations.append({
#             "image": image_path,
#             "text": text,
#             "words": words
#         })

#     # Simpan hasil dalam file JSON
#     with open(output_file, "w", encoding="utf-8") as f:
#         json.dump(annotations, f, indent=4, ensure_ascii=False)


# def convert_dataset_inter(dataset, output_file):
#     """
#     Mengonversi dataset faktur/invoice ke format JSON untuk EasyOCR.
#     """
#     annotations = []

#     for i, data in enumerate(dataset):
#         image = data["image"]  # PIL Image
#         ground_truth = json.loads(data["ground_truth"])["gt_parse"]  # Parse JSON

#         # Simpan gambar
#         image_path = f"dataset_images/invoice_{i}.jpg"
#         os.makedirs("dataset_images", exist_ok=True)
#         image.save(image_path)

#         words = []
#         items = ground_truth.get("items", [])  # Pastikan mengambil 'items' dengan default []
#         if items:
#             for item in items:
#                 words.append({
#                     "text": item.get("item_desc", "UNKNOWN_ITEM"),
#                     "quad": None,
#                     "row_id": None,
#                     "group_id": None
#                 })

#         annotations.append({
#             "image": image_path,
#             "text": ground_truth,  # Simpan seluruh teks yang diparsing
#             "bbox": words
#         })

#         # Pastikan folder Google Drive tujuan benar
#         drive_folder = "G:/My Drive/dataset"
#         output_file = os.path.join(drive_folder, "dataset_annotations.json")

#         # Simpan sebagai JSON
#         with open(output_file, "w", encoding="utf-8") as f:
#             json.dump(annotations, f, indent=4, ensure_ascii=False)

#         print(f"Dataset invoice berhasil disimpan ke {output_file}")


In [ ]:
# # Konversi dataset 1 dan dataset 2
# convert_dataset_local(data_tuning_local, "data_tuning_local.json")
# convert_dataset_inter(data_tuning_inter, "data_tuning_inter.json")

In [ ]:
# python train.py --batch_size 16 --num_epochs 10 --lr 0.001 --train_data data_tuning_local.json data_tuning_inter.json


POST-PROCESSING

Pre-Processing Hasil Text Extraction

Training BERT/NER